In [1]:
import requests
import PyPDF2

In [2]:
def read_text_file(name) :
    df=open('note.txt','r')
    return df.read()
def add_2_numbers(a,b):
    return (float(a)+float(b))
def calc(num):
    return ((float(num)*(float(num)-1))/2)
def get_pdf_parameters(ur1: str,ur2: str,kpi: str,quarter: str):
    return {
        "url1": ur1,
        "url2": ur2,
        "key_metric": kpi,
        "quarter": quarter
    }


In [3]:
function_dict={'read_text_file':read_text_file,'add_2_numbers':add_2_numbers,'calc':calc,'get_pdf_parameters':get_pdf_parameters}

In [4]:
fn1= {
    'type':'function',
    "function": {
        "name": "read_text_file",
        "description": "Reads a notepad file and returns its contents",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The full path to the notepad file"
                }
            },
            "required": ["name"]
        }
    }
}

fn2= {
    'type':'function',
    "function": {
        "name": "add_2_numbers",
        "description": "Adds 2 numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "float",
                    "description": "First number to be added"
                },
                "b": {
                    "type": "float",
                    "description": "Second number to be added"
                }
            },
            "required": ["a","b"]
        }
    }
}
fn3= {
    'type':'function',
    "function": {
        "name": "calc",
        "description": "Does a mathematical calculation",
        "parameters": {
            "type": "object",
            "properties": {
                "num": {
                    "type": "float",
                    "description": "Number to which calculation needs to be done"
                },
                
            },
            "required": ['num']
        }
    }
}
 
fn5 = {
    "type": "function",
    "function": {
        "name": "get_pdf_parameters",
        "description": "Extracts PDF file paths and the key metric to search from the user query",
        "parameters": {
            "type": "object",
            "properties": {
                "ur1": {
                    "type": "string",
                    "description": "Provide the document 1 that needs to be read"
                },
                "ur2": {
                    "type": "string",
                    "description": "Provide the document 2 that needs to be read"
                },
                "kpi": {
                    "type": "string",
                    "description": "Provide the kpi that needs to be extracted"
                },
                "quarter": {
                    "type": "string",
                    "description": "Provide the quater for which you need the data "
                }
            },
            "required": ["ur1","ur2","kpi","quarter"]
        }
    }
}

In [5]:
OLLAMA_URL = "http://localhost:11434/api/chat"
url=OLLAMA_URL
data={
    "model":"llama3.2:3b",
    "messages":[
        {
            'role':'user',
            'content':'what is the net income for 2025Q1 from C:/Users/Akshaya V/Downloads/citi.pdf and C:/Users/Akshaya V/Downloads/jpmc.pdf pdf files' ,
        }],
    "stream":False,
    "tools":[fn1,fn2,fn3,fn5]
}
r=requests.post(url,json=data)
r.json()


{'model': 'llama3.2:3b',
 'created_at': '2025-06-14T11:20:29.5000553Z',
 'message': {'role': 'assistant',
  'content': '',
  'tool_calls': [{'function': {'name': 'get_pdf_parameters',
     'arguments': {'kpi': 'net income',
      'quarter': '2025Q1',
      'ur1': 'C:/Users/Akshaya V/Downloads/citi.pdf',
      'ur2': 'C:/Users/Akshaya V/Downloads/jpmc.pdf'}}}]},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 352817971900,
 'load_duration': 80775300,
 'prompt_eval_count': 454,
 'prompt_eval_duration': 167707926700,
 'eval_count': 58,
 'eval_duration': 185026630000}

In [6]:
f_name=r.json()['message']['tool_calls'][0]['function']['name']
f_args=r.json()['message']['tool_calls'][0]['function']['arguments']
print(f_name)
print(f_args)
print(f_args['kpi'])

get_pdf_parameters
{'kpi': 'net income', 'quarter': '2025Q1', 'ur1': 'C:/Users/Akshaya V/Downloads/citi.pdf', 'ur2': 'C:/Users/Akshaya V/Downloads/jpmc.pdf'}
net income


In [7]:
print("Please extract and return the value of " +str(f_args['kpi'])+" from each document for the quarter "+str(f_args['quarter'])+ " in JSON format.")

Please extract and return the value of net income from each document for the quarter 2025Q1 in JSON format.


In [25]:
def pdf_to_text(path):
    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        return "\n".join(page.extract_text() or "" for page in reader.pages)

In [28]:
pdf1_text = pdf_to_text(str(f_args['ur1']))
pdf2_text = pdf_to_text(str(f_args['ur2']))
pdf=(pdf1_text+pdf2_text)

In [10]:
 prompt = f"""
    **Task:** Extract {f_args['kpi']} for {f_args['quarter']} from two bank reports.
    The metric may appear in different forms across documents:
    - Short form (e.g., "EPS") 
    - Full form (e.g., "Earnings Per Share")
    - Common variations (e.g., "Diluted EPS", "Basic EPS" for EPS)


    ---

    **JPMorgan Chase (JPMC) Report:**
    {pdf1_text}

    ---

    **Citigroup (Citi) Report:**
    {pdf2_text}

    ---

    **Rules:**
    1. Search for {f_args['quarter']} (accept equivalents: "Q1 2025", "first quarter", "Jan-Mar 2025")
    2. Recognize all forms of the KPI (e.g., both "EPS" and "Earnings Per Share")
    3. Focus on financial highlights and tables first
    4. Return "not found" if unavailable

    **Output Format (JSON):**
    {{
      "jpmc": "<value or 'not found'>",
      "citi": "<value or 'not found'>"
    }}
    
    Key improvements:
    
    1. **Term Variation Handling**:
       - Explicitly mentions short/full forms (EPS vs Earnings Per Share)
       - Includes common financial variants (Diluted/Basic EPS)
    
    2. **Clear Examples**:
       - Shows exactly how different formats should be parsed
       - Demonstrates quarter name variations
    
    3. **Enhanced Output**:
       - Added "unit" field to capture currency consistency
       - Strict JSON formatting requirement
    
    4. **Processing Guidance**:
       - Prioritizes financial highlights/tables
       - Handles different currency symbols
       - Explains conflict resolution for multiple matches
    
    Example Outputs:
    
    1. When KPI is found in both:
    ```json
    {{
      "<auto_bank1>": "<value>",
      "<auto_bank2>": "<value>"
    }}```
    """

In [11]:
print(prompt)


   **Task:** Extract net income for 2025Q1 from two bank reports.
   The metric may appear in different forms across documents:
   - Short form (e.g., "EPS") 
   - Full form (e.g., "Earnings Per Share")
   - Common variations (e.g., "Diluted EPS", "Basic EPS" for EPS)


   ---

   **JPMorgan Chase (JPMC) Report:**
    
  1 
 Citi CEO Jane Fraser said, “With 
net income of $4.1 billion we 
delivered a strong quarter, 
marked by continued 
momentum, positive operating 
leverage and improved returns 
in each of our five businesses. 
Services recorded its best first 
quarter revenue in a decade. 
Markets had a good first quarter 
with revenue up 12% driven by 
strong client activity and 
monetization. Banking was up 
12% with M&A revenue nearly 
double from what it was last 
year. Wealth revenues increased 
24% with progress across all 
three client segments. USPB 
was up 2%, driven mainly by 
growth in Branded Cards, and 
also saw improved returns. We 
returned $2.8 billion in capital 
t

In [17]:
final_data = {
    "model": "llama3.2:3b",
    "messages": [
        {
            "role": "system",
            "content": "You are a bank document processor that automatically identifies financial institutions and extracts data."
        },
        {
            "role": "user",
            "content": prompt
        }
    ],
    "response_format": {
        "type": "json_object",
        "schema": {
            "type": "object",
            "patternProperties": {
                "^[A-Z]{1,5}$": {  # Matches bank abbreviations (1-5 chars)
                    "type": ["string", "null"]
                }
            },
            "additionalProperties": False
        }
    },
    "temperature": 0.1,
    "stream": False
}
final_response = requests.post(OLLAMA_URL, json=final_data)
print(final_response.json())


{'model': 'llama3.2:3b', 'created_at': '2025-06-14T11:47:32.7449744Z', 'message': {'role': 'assistant', 'content': "Here are the extracted financial highlights and tables from JPMorgan Chase & Co.'s (JPM) 2025Q1 news release:\n\n**Financial Highlights**\n\n* Net income: $14.6 billion, up 9% from Q1 2024\n* Net revenue: $46.0 billion, up 8%\n* Earnings per share (EPS): $5.07, up 5% from Q1 2024\n\n**Tables**\n\n**Net Revenue by Business Segment**\n\n| Business Segment | Net Revenue - Reported | Net Revenue - Managed |\n| --- | --- | --- |\n| Consumer & Community Banking (CCB) | $18.3 billion | $18.6 billion |\n| Corporate and Other | $2.8 billion | $3.2 billion |\n| Markets | $9.7 billion | $10.0 billion |\n| Wholesale Banking | $16.2 billion | $17.1 billion |\n\n**Noninterest Expense by Business Segment**\n\n| Business Segment | Noninterest Expense - Reported |\n| --- | --- |\n| Consumer & Community Banking (CCB) | $9.9 billion |\n| Corporate and Other | $3.6 billion |\n| Markets | $5.

In [13]:

# final_response = requests.post(
#     OLLAMA_URL,
#     json=final_data,
#     timeout=120
# )
# final_response.raise_for_status()  # Raises HTTPError for bad responses

# # 4. Process and validate the response
# response_data = final_response.json()

# if "message" in response_data and "content" in response_data["message"]:
#     try:
#         extracted_data = json.loads(response_data["message"]["content"])
#         print("Extracted Data:")
#         print(json.dumps(extracted_data, indent=2))
#     except json.JSONDecodeError:
#         print("Error: Invalid JSON received")
#         print("Raw response:", response_data["message"]["content"])
# else:
#     print("Unexpected response format:", response_data)



In [14]:
# print(response_data)


In [15]:

followup_messages = [
    {"role": "system", "content": "You are an expert analyst in analysing financial data"},
    {"role": "user", "content": prompt}]

final_data = {
    "model": "llama3.2:3b",
    "messages": followup_messages,
    "temperature": 0.1,  # Lower for less creativity
    "response_format": {"type": "json_object"}, 
    "stream": False,
    "format":{
        "type":"object",
        "properties":{
            "KPI bank1":{
                "type":"string"},
        "KPI bank2":{
                "type":"string"}},"required":["KPI bank1","KPI bank2"]},
}

final_response = requests.post(OLLAMA_URL, json=final_data)
print(final_response.json())


{'model': 'llama3.2:3b', 'created_at': '2025-06-14T11:28:57.4990859Z', 'message': {'role': 'assistant', 'content': '{\n"KPI bank1": "14,643",\n"KPI bank2": "14,643"\n}\n   \t\t\t\t        \t\t'}, 'done_reason': 'stop', 'done': True, 'total_duration': 505541722900, 'load_duration': 91639200, 'prompt_eval_count': 4096, 'prompt_eval_duration': 497374523900, 'eval_count': 32, 'eval_duration': 8028755400}


In [16]:

followup_messages = [
    {"role": "system", "content": "You are an expert analyst in analysing financial data"},
    {"role": "user", "content": prompt}]

final_data = {
    "model": "llama3.2:3b",
    "messages": followup_messages,
    "temperature": 0.1,  # Lower for less creativity
    "response_format": {"type": "json_object"}, 
    "stream": False,
    
}

final_response = requests.post(OLLAMA_URL, json=final_data)
print(final_response.json())


{'model': 'llama3.2:3b', 'created_at': '2025-06-14T11:37:53.8966806Z', 'message': {'role': 'assistant', 'content': 'Here is the output in JSON format, following the specified rules and improvements:\n\n```json\n{\n  "jpmc": {\n    "EPS": 5.07,\n    "Earnings Per Share": 5.07,\n    "Diluted EPS": 5.07,\n    "Basic EPS": 4.81,\n    "Net Income": 14,643,\n    "Net Revenue": 46,014,\n    "Noninterest Expense": 23,597\n  },\n  "citi": {\n    "EPS": null,\n    "Earnings Per Share": null,\n    "Diluted EPS": null,\n    "Basic EPS": null,\n    "Net Income": null,\n    "Net Revenue": null,\n    "Noninterest Expense": null\n  }\n}\n```\n\nNote: I\'ve included the `EPS` and `Earnings Per Share` variants as they were mentioned in the rules. If you want to prioritize one over the other, please let me know.\n\nFor the Citigroup section, since no KPI was found, it remains as `null`.\n\nIf you\'d like to include additional information or format changes, please feel free to specify!'}, 'done_reason': '

In [18]:

followup_messages = [
    {"role": "system", "content": "You are an expert analyst in analysing financial data"},
    {"role": "user", "content": prompt}]

final_data = {
    "model": "llama3.2:3b",
    "messages": followup_messages,
    "response_format": {"type": "json_object"}, 
    "stream": False,
    
}

final_response = requests.post(OLLAMA_URL, json=final_data)
print(final_response.json())


{'model': 'llama3.2:3b', 'created_at': '2025-06-14T11:59:22.1226232Z', 'message': {'role': 'assistant', 'content': 'Here are the financial highlights and tables from JPMorgan Chase & Co.\'s 2025Q1 news release, along with Citi\'s:\n\n**JPMorgan Chase & Co.**\n\n* Net income was $14.6 billion, up 9%.\n* Net revenue was $46.0 billion, up 8%.\n* Noninterest expense was $23.6 billion, up 4%.\n\n**Citi**\n\n* Net income was $14.643 billion, up 5% ($638 million).\n* Earnings Per Share (EPS) was $5.07, up 5% ($0.26).\n* Return on common equity was 18%.\n* Return on tangible common equity was 21.\n\nHere is the output in JSON format:\n\n```json\n{\n  "jpmc": {\n    "net_income": "$14.6B",\n    "net_revenue": "$46.0B",\n    "noninterest_expense": "$23.6B"\n  },\n  "citi": {\n    "eps": "$5.07",\n    "return_on_common_equity": "18%",\n    "return_on_tangible_common_equity": "21%"\n  }\n}\n```\n\nPlease note that I\'ve kept the currency symbols in the output to maintain consistency, but it\'s alw

In [22]:
import requests
import json

# Debugged and improved version
followup_messages = [
    {
        "role": "system", 
        "content": "You are an expert financial analyst that outputs strictly formatted JSON data."
    },
    {
        "role": "user", 
        "content": prompt
    }
]

# Create dynamic property names
kpi_url1 = f"{f_args['kpi']} for {f_args['ur1']}"
kpi_url2 = f"{f_args['kpi']} for {f_args['ur2']}"

final_data = {
    "model": "llama3.2:3b",
    "messages": followup_messages,
    "response_format": {
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {
                kpi_url1: {"type": "string"},
                kpi_url2: {"type": "string"}
            },
            "required": [kpi_url1, kpi_url2],
            "additionalProperties": False
        }
    },
    "stream": False
}

try:
    # Make the API request
    final_response = requests.post(
        OLLAMA_URL,
        json=final_data,
        timeout=120
    )
    final_response.raise_for_status()  # Check for HTTP errors
    
    # Process and validate the response
    response_json = final_response.json()
    
    if "message" in response_json and "content" in response_json["message"]:
        try:
            # Parse and pretty-print the JSON response
            parsed_data = json.loads(response_json["message"]["content"])
            print(json.dumps(parsed_data, indent=2))
        except json.JSONDecodeError:
            print("Error: Invalid JSON received")
            print("Raw response:", response_json["message"]["content"])
    else:
        print("Unexpected response format:", response_json)

except requests.exceptions.RequestException as e:
    print(f"API request failed: {str(e)}")
except Exception as e:
    print(f"Unexpected error: {str(e)}")

API request failed: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=120)


In [ ]:
## providing one merged file (jpmc+citi) as input instead of 2 sepearte text files 

In [29]:
 prompt = f"""
    **Task:** Extract {f_args['kpi']} for {f_args['quarter']} from two bank reports.
    The metric may appear in different forms across documents:
    - Short form (e.g., "EPS") 
    - Full form (e.g., "Earnings Per Share")
    - Common variations (e.g., "Diluted EPS", "Basic EPS" for EPS)


    ---

    **Document**
    {pdf}

    ---

    **Rules:**
    1. Search for {f_args['quarter']} (accept equivalents: "Q1 2025", "first quarter", "Jan-Mar 2025")
    2. Recognize all forms of the KPI (e.g., both "EPS" and "Earnings Per Share")
    3. Focus on financial highlights and tables first
    4. Return "not found" if unavailable

    **Output Format (JSON):**
    {{
      "jpmc": "<value or 'not found'>",
      "citi": "<value or 'not found'>"
    }}
    
    Key improvements:
    
    1. **Term Variation Handling**:
       - Explicitly mentions short/full forms (EPS vs Earnings Per Share)
       - Includes common financial variants (Diluted/Basic EPS)
    
    2. **Clear Examples**:
       - Shows exactly how different formats should be parsed
       - Demonstrates quarter name variations
    
    3. **Enhanced Output**:
       - Added "unit" field to capture currency consistency
       - Strict JSON formatting requirement
    
    4. **Processing Guidance**:
       - Prioritizes financial highlights/tables
       - Handles different currency symbols
       - Explains conflict resolution for multiple matches
    
    Example Outputs:
    
    1. When KPI is found in both:
    ```json
    {{
      "<auto_bank1>": "<value>",
      "<auto_bank2>": "<value>"
    }}```
    """

In [30]:
print(prompt[:400])


   **Task:** Extract net income for 2025Q1 from two bank reports.
   The metric may appear in different forms across documents:
   - Short form (e.g., "EPS") 
   - Full form (e.g., "Earnings Per Share")
   - Common variations (e.g., "Diluted EPS", "Basic EPS" for EPS)


   ---

   **Document**
    
  1 
 Citi CEO Jane Fraser said, “With 
net income of $4.1 billion we 
delivered a strong quarter, 


In [ ]:

followup_messages = [
    {"role": "system", "content": "You are an expert analyst in analysing financial data"},
    {"role": "user", "content": prompt}]

final_data = {
    "model": "llama3.2:3b",
    "messages": followup_messages,
    "response_format": {"type": "json_object"}, 
    "stream": False,
    
}

final_response = requests.post(OLLAMA_URL, json=final_data)
print(final_response.json())
